# 1. Inspección Inicial del Dataset

Este paso revisa estructura, muestra, tipos de datos, nulos, duplicados y estadísticas generales del archivo antes de limpieza o EDA profundo.


In [ ]:
# Importaciones (inicio del archivo)
import os
import sys
import re
import math
import json
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
import unicodedata


In [ ]:
# Cargar el dataset (asegúrate de que el archivo está en el mismo directorio del notebook)
df = pd.read_csv("../data/raw/datos_raw_startups_2025.csv")

# Dimensiones del dataset
print("Dimensiones:", df.shape)


Dimensiones: (1000, 12)


In [8]:
# Vista de los primeros registros
print("\nPrimeras filas:")
df.head(10)


Primeras filas:


,sector,monto_financiado,num_rondas,tamano_equipo,exp_fundadores,presencia_redes,ubicacion,inversores_destacados,descripcion,tiempo_fundacion,estado_operativo,viabilidad
0,Salud,4287772.73,3,8,8,2,San Francisco,0,Análisis predictivo para diagnóstico temprano....,3,en operación,1
1,Fintech,375259.30,1,11,16,11,Londres,0,Pagos instantáneos con KYC automatizado. de rá...,8,en expansión,0
2,Salud,988308.41,1,4,3,6,Los Ángeles,0,Wearables para monitoreo cardiaco continuo.,1,en operación,0
3,Tecnología,1967712.54,3,2,8,9,Nueva York,1,IA para optimizar procesos empresariales. con ...,3,en operación,0
4,Agricultura,978833.75,1,19,9,4,Los Ángeles,0,Sensórica de cultivos con riego automatizado. ...,10,en expansión,1
5,Fintech,114897.68,0,4,6,0,Boston,0,Infraestructura de open banking para pymes. co...,1,idea,0
6,Transporte,8633202.93,6,21,7,3,Los Ángeles,0,Gestión de flotas con telemetría avanzada.,4,en expansión,1
7,E-commerce,576609.20,4,4,7,24,Barcelona,0,Checkout unificado con antifraude. listo para ...,6,en operación,1
8,E-commerce,1105033.27,0,3,6,2,Berlín,1,Checkout unificado con antifraude. con enfoque...,0,idea,1
9,Salud,1158745.89,1,9,10,0,Barcelona,0,Telemedicina con historial clínico interoperab...,1,en operación,0


In [10]:
# Tipos de datos por columna
print("\nTipos de datos:")
df.dtypes


Tipos de datos:


sector                    object
monto_financiado         float64
num_rondas                 int64
tamano_equipo              int64
exp_fundadores             int64
presencia_redes            int64
ubicacion                 object
inversores_destacados      int64
descripcion               object
tiempo_fundacion           int64
estado_operativo          object
viabilidad                 int64
dtype: object

In [11]:
# Conteo de valores nulos por columna
print("\nValores nulos por columna:")
df.isnull().sum()


Valores nulos por columna:


sector                   0
monto_financiado         0
num_rondas               0
tamano_equipo            0
exp_fundadores           0
presencia_redes          0
ubicacion                0
inversores_destacados    0
descripcion              0
tiempo_fundacion         0
estado_operativo         0
viabilidad               0
dtype: int64

In [12]:
# Cantidad de filas duplicadas
print("\nCantidad de registros duplicados:", df.duplicated().sum())


Cantidad de registros duplicados: 0


In [14]:
# Descriptivo general (numéricas)
print("\nEstadísticas numéricas:")
df.describe()


Estadísticas numéricas:


,monto_financiado,num_rondas,tamano_equipo,exp_fundadores,presencia_redes,inversores_destacados,tiempo_fundacion,viabilidad
count,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,3.114103e+06,1.946000,9.941000,6.278000,5.846000,0.259000,3.887000,0.64000
std,3.415882e+06,1.640663,5.715609,3.658429,7.572425,0.438305,2.786395,0.48024
min,1.000000e+04,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,4.981217e+05,1.000000,5.000000,4.000000,1.000000,0.000000,2.000000,0.00000
50%,1.496787e+06,2.000000,9.000000,6.000000,3.000000,0.000000,4.000000,1.00000
75%,4.805616e+06,3.000000,14.000000,8.000000,7.000000,1.000000,6.000000,1.00000
max,1.000000e+07,7.000000,25.000000,19.000000,67.000000,1.000000,15.000000,1.00000


In [16]:
# Descriptivo para categóricas
print("\nEstadísticas categóricas:")
df.describe(include=['object']).T


Estadísticas categóricas:


,count,unique,top,freq
sector,1000,10,Salud,112
ubicacion,1000,10,Boston,117
descripcion,1000,444,Créditos P2P con scoring alternativo.,7
estado_operativo,1000,4,en operación,479


# 2. Limpieza y formateo básico
Este paso estandariza nombres, normaliza categorías, crea variables derivadas y valida rangos; no se eliminan registros porque no hay nulos ni duplicados detectados en la inspección inicial.

## 2.1 Duplicado: 
Crear df1 = df.copy(deep=True); todas las modificaciones van a df1.

In [18]:
# 2.1 Duplicado
df1 = df.copy(deep=True)  # trabajar SOLO sobre df1

## 2.2 Columnas:
Estandarizar a snake_case, minúsculas y sin tildes/espacios; conservar nombres existentes ya normalizados como en el archivo.

In [19]:
# 2.2 Estandarizar columnas
df1.columns = (
    df1.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_", regex=False)
)

## 2.3 Tipos: 
monto_financiado float; num_rondas, tamano_equipo, exp_fundadores, presencia_redes, tiempo_fundacion, inversores_destacados, viabilidad como int; sector, ubicacion, estado_operativo como categóricas/strings.

In [20]:
# 2.3 Tipos de datos
numeric_int = ["num_rondas","tamano_equipo","exp_fundadores","presencia_redes","tiempo_fundacion","inversores_destacados","viabilidad"]
for c in numeric_int:
    if c in df1.columns:
        df1[c] = pd.to_numeric(df1[c], errors="coerce").astype("Int64")

if "monto_financiado" in df1.columns:
    df1["monto_financiado"] = pd.to_numeric(df1["monto_financiado"], errors="coerce").astype(float)

for c in ["sector","ubicacion","estado_operativo"]:
    if c in df1.columns:
        df1[c] = df1[c].astype("string").str.strip().str.lower()

## 2.4 Texto/categorías: 
Limpiar descripcion (strip, minúsculas, espacios, tildes opcional); unificar estado_operativo a {en operacion, en expansion, idea, cerrada}; revisar variantes en sector y ubicacion.

In [23]:
# 2.4 Limpieza de texto/categorías
if "descripcion" in df1.columns:
    s = df1["descripcion"].astype("string").fillna("")
    s = s.str.strip().str.lower()
    s = s.str.replace(r"\\s+", " ", regex=True)
    s = s.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii','ignore').decode('ascii'))
    df1["descripcion"] = s

estado_map = {
    "en operacion": "en operacion",
    "en operación": "en operacion",
    "en expansion": "en expansion",
    "en expansión": "en expansion",
    "idea": "idea",
    "cerrada": "cerrada"
}
if "estado_operativo" in df1.columns:
    df1["estado_operativo"] = df1["estado_operativo"].map(lambda x: estado_map.get(x, x))

## 2.5 Rangos/outliers: 
Asegurar no negativos en métricas básicas; crear flags IQR para outliers de monto_financiado, tamano_equipo, presencia_redes sin eliminar aún.

In [24]:
# 2.5 Flags de outliers (IQR)
for c in ["monto_financiado","tamano_equipo","presencia_redes"]:
    if c in df1.columns:
        q1, q3 = df1[c].quantile([0.25, 0.75])
        iqr = q3 - q1
        low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
        df1[f"is_outlier_{c}"] = ((df1[c] < low) | (df1[c] > high)).astype("Int64")

## 2.6 Derivadas: 
log_monto = log1p(monto_financiado); intensidad_redes = min(presencia_redes, 50)/50; antiguedad_bucket por cortes {0-1, 2-3, 4-7, 8+}; sector_x_estado como interacción; geo_region opcional por mapeo de ubicacion.

In [25]:
# 2.6 Variables derivadas
if "monto_financiado" in df1.columns:
    df1["log_monto"] = np.log1p(df1["monto_financiado"])

if "presencia_redes" in df1.columns:
    capped = np.minimum(df1["presencia_redes"].astype(float), 50.0)
    df1["intensidad_redes"] = (capped / 50.0).round(4)

if "tiempo_fundacion" in df1.columns:
    df1["antiguedad"] = df1["tiempo_fundacion"].astype(float)
    bins = [-np.inf, 1, 3, 7, np.inf]
    labels = ["0-1","2-3","4-7","8+"]
    df1["antiguedad_bucket"] = pd.cut(df1["antiguedad"], bins=bins, labels=labels)

if set(["sector","estado_operativo"]).issubset(df1.columns):
    df1["sector_x_estado"] = df1["sector"].astype("string") + "__" + df1["estado_operativo"].astype("string")

## 2.7 Calidad: 
Imputación simple de NA (mediana en numéricas, “desconocido” en categóricas) o dejar NA si se prefiere imputar en el pipeline; revisar cardinalidad de categorías y balance de viabilidad.

In [26]:
# 2.7 Calidad (imputación simple opcional)
for c in numeric_int + ["monto_financiado"]:
    if c in df1.columns and df1[c].isna().any():
        df1[c] = df1[c].fillna(df1[c].median())

for c in ["sector","ubicacion","estado_operativo","descripcion"]:
    if c in df1.columns:
        df1[c] = df1[c].fillna("desconocido")

## 2.8 Persistencia: 
Crear data/processed y guardar startups_sintetico_1000_processed.csv y .parquet; a partir de aquí, todo EDA/modelado debe leer desde data/processed.

In [28]:
# 2.8 Persistencia en data/processed
PROJECT_ROOT = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

processed_csv_path = PROCESSED_DIR / 'startups_sintetico_1000_processed.csv'
processed_parquet_path = PROCESSED_DIR / 'startups_sintetico_1000_processed.parquet'

# Guardar
df1.to_csv(processed_csv_path, index=False)
try:
    df1.to_parquet(processed_parquet_path, index=False)
except Exception:
    print('Aviso: instala pyarrow o fastparquet para escribir Parquet. Se guardó CSV.')

print('Guardado:')
print(' -', processed_csv_path)
print(' -', processed_parquet_path)

Guardado:
 - c:\Users\carlo\OneDrive\Escritorio\Proyecto Machine Learning\data\processed\startups_sintetico_1000_processed.csv
 - c:\Users\carlo\OneDrive\Escritorio\Proyecto Machine Learning\data\processed\startups_sintetico_1000_processed.parquet


# 2. EDA
## Objetivo: 
Explorar distribuciones, relaciones y calidad del dataset procesado sin modificarlo; generar resúmenes, correlaciones, tablas cruzadas y agregaciones por segmento para orientar hipótesis y modelado.

## Checklist: 
Confirmar existencia de data/processed/startups_sintetico_1000_processed.(parquet|csv), revisar tipos y balance de la variable objetivo viabilidad antes de graficar .

## Entregables: 
CSVs con resumen numérico, frecuencias categóricas, matriz de correlación Spearman, tablas cruzadas viabilidad x categorías, agregados por segmento y métricas de calidad de datos.

In [36]:
# 3.1 Cargar dataset procesado desde la raíz del proyecto
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
parquet = PROCESSED_DIR / 'startups_sintetico_1000_processed.parquet'
csv = PROCESSED_DIR / 'startups_sintetico_1000_processed.csv'

dfp = pd.read_parquet(parquet) if parquet.exists() else pd.read_csv(csv)
print(f"Dataset procesado cargado: {dfp.shape} filas x {dfp.shape[1]} columnas")

Dataset procesado cargado: (1000, 20) filas x 20 columnas


In [37]:
# 3.2 Distribuciones numéricas
num_cols = [
    'monto_financiado','num_rondas','tamano_equipo','exp_fundadores',
    'presencia_redes','tiempo_fundacion','log_monto','intensidad_redes'
]
num_cols = [c for c in num_cols if c in dfp.columns]
summary = dfp[num_cols].describe(percentiles=[0.25,0.5,0.75]).T
print("\nResumen numérico (describe):")
display(summary)


Resumen numérico (describe):


,count,mean,std,min,25%,50%,75%,max
monto_financiado,1000.0,3114103.13917,3415881.565779,10000.0,498121.6625,1496787.465,4805615.905,10000000.0
num_rondas,1000.0,1.946,1.640663,0.0,1.0,2.0,3.0,7.0
tamano_equipo,1000.0,9.941,5.715609,1.0,5.0,9.0,14.0,25.0
exp_fundadores,1000.0,6.278,3.658429,0.0,4.0,6.0,8.0,19.0
presencia_redes,1000.0,5.846,7.572425,0.0,1.0,3.0,7.0,67.0
tiempo_fundacion,1000.0,3.887,2.786395,0.0,2.0,4.0,6.0,15.0
log_monto,1000.0,14.171127,1.401963,9.21044,13.118601,14.218827,15.385296,16.118096
intensidad_redes,1000.0,0.11658,0.149063,0.0,0.02,0.06,0.14,1.0


In [38]:
# 3.3 Distribuciones categóricas
cat_cols = [c for c in ['sector','ubicacion','estado_operativo','antiguedad_bucket'] if c in dfp.columns]
for c in cat_cols:
    freq = dfp[c].value_counts(dropna=False)
    prop = (dfp[c].value_counts(normalize=True, dropna=False)*100).round(2)
    out = pd.concat([freq.rename('count'), prop.rename('pct')], axis=1)
    print(f"\nFrecuencias de {c}:")
    display(out)


Frecuencias de sector:


,count,pct
sector,,
salud,112,11.2
energía renovable,112,11.2
fintech,106,10.6
e-commerce,104,10.4
entretenimiento,104,10.4
turismo,102,10.2
transporte,101,10.1
educación,90,9.0
tecnología,87,8.7



Frecuencias de ubicacion:


,count,pct
ubicacion,,
boston,117,11.7
ciudad de méxico,112,11.2
berlín,105,10.5
nueva york,105,10.5
são paulo,103,10.3
los ángeles,95,9.5
san francisco,94,9.4
londres,92,9.2
barcelona,92,9.2



Frecuencias de estado_operativo:


,count,pct
estado_operativo,,
en operacion,479,47.9
en expansion,281,28.1
idea,127,12.7
cerrada,113,11.3



Frecuencias de antiguedad_bucket:


,count,pct
antiguedad_bucket,,
4-7,424,42.4
2-3,245,24.5
0-1,230,23.0
8+,101,10.1


In [39]:
# 3.4 Correlaciones (Spearman por robustez)
if num_cols:
    corr = dfp[num_cols].corr(method='spearman')
    print("\nMatriz de correlación (Spearman):")
    display(corr)


Matriz de correlación (Spearman):


,monto_financiado,num_rondas,tamano_equipo,exp_fundadores,presencia_redes,tiempo_fundacion,log_monto,intensidad_redes
monto_financiado,1.000000,0.779258,0.388496,0.186166,0.097196,0.364087,1.000000,0.097191
num_rondas,0.779258,1.000000,0.400145,0.224690,0.157470,0.395876,0.779258,0.157465
tamano_equipo,0.388496,0.400145,1.000000,0.325773,-0.011639,0.412913,0.388496,-0.011643
exp_fundadores,0.186166,0.224690,0.325773,1.000000,0.008576,0.251436,0.186166,0.008569
presencia_redes,0.097196,0.157470,-0.011639,0.008576,1.000000,0.051864,0.097196,1.000000
tiempo_fundacion,0.364087,0.395876,0.412913,0.251436,0.051864,1.000000,0.364087,0.051855
log_monto,1.000000,0.779258,0.388496,0.186166,0.097196,0.364087,1.000000,0.097191
intensidad_redes,0.097191,0.157465,-0.011643,0.008569,1.000000,0.051855,0.097191,1.000000


In [40]:
# 3.4 Tablas cruzadas: viabilidad por categorías
if 'viabilidad' in dfp.columns:
    for c in ['sector','ubicacion','estado_operativo']:
        if c in dfp.columns:
            ct = pd.crosstab(dfp[c], dfp['viabilidad'], normalize='index').round(3)
            print(f"\nViabilidad por {c} (proporciones por fila):")
            display(ct)


Viabilidad por sector (proporciones por fila):


viabilidad,0,1
sector,,
agricultura,0.378,0.622
e-commerce,0.327,0.673
educación,0.389,0.611
energía renovable,0.375,0.625
entretenimiento,0.375,0.625
fintech,0.387,0.613
salud,0.357,0.643
tecnología,0.379,0.621
transporte,0.376,0.624



Viabilidad por ubicacion (proporciones por fila):


viabilidad,0,1
ubicacion,,
barcelona,0.413,0.587
berlín,0.343,0.657
boston,0.316,0.684
ciudad de méxico,0.348,0.652
londres,0.315,0.685
los ángeles,0.316,0.684
nueva york,0.352,0.648
san francisco,0.457,0.543
são paulo,0.340,0.660



Viabilidad por estado_operativo (proporciones por fila):


viabilidad,0,1
estado_operativo,,
cerrada,0.876,0.124
en expansion,0.189,0.811
en operacion,0.284,0.716
idea,0.567,0.433


In [41]:
# 3.5 Análisis por segmentos (sector x estado_operativo)
by_cols = [c for c in ['sector','estado_operativo'] if c in dfp.columns]
if by_cols:
    seg = dfp.groupby(by_cols, as_index=False).agg({
        'viabilidad':'mean' if 'viabilidad' in dfp.columns else 'size',
        'log_monto':'median' if 'log_monto' in dfp.columns else 'size',
        'tamano_equipo':'mean' if 'tamano_equipo' in dfp.columns else 'size',
        'presencia_redes':'mean' if 'presencia_redes' in dfp.columns else 'size'
    })
    seg = seg.rename(columns={
        'viabilidad':'viabilidad_mean',
        'log_monto':'log_monto_median',
        'tamano_equipo':'tamano_equipo_mean',
        'presencia_redes':'presencia_redes_mean'
    })
    if 'viabilidad_mean' in seg.columns:
        seg = seg.sort_values('viabilidad_mean', ascending=False)
    print("\nSegmentos (sector x estado_operativo):")
    display(seg)


Segmentos (sector x estado_operativo):


,sector,estado_operativo,viabilidad_mean,log_monto_median,tamano_equipo_mean,presencia_redes_mean
1,agricultura,en expansion,0.894737,15.309240,15.421053,6.947368
25,salud,en expansion,0.888889,15.442195,16.25,5.083333
38,turismo,en operacion,0.836735,14.755174,7.755102,7.285714
17,entretenimiento,en expansion,0.833333,15.835383,16.5,8.833333
22,fintech,en operacion,0.818182,14.019611,9.0,4.840909
21,fintech,en expansion,0.814815,15.387945,16.111111,5.185185
37,turismo,en expansion,0.814815,15.373446,14.481481,8.481481
13,energía renovable,en expansion,0.8,15.336921,15.533333,4.866667
9,educación,en expansion,0.8,15.037450,15.52,5.76
29,tecnología,en expansion,0.791667,15.765081,16.625,8.625


In [42]:
# 3.6 Calidad de datos
na_pct = (dfp.isna().mean()*100).round(2).rename('pct_na').to_frame()
print("\nPorcentaje de NA por columna:")
display(na_pct)

outlier_cols = [c for c in dfp.columns if c.startswith('is_outlier_')]
if outlier_cols:
    out_counts = dfp[outlier_cols].sum().rename('count')
    print("\nConteo de flags de outliers:")
    display(out_counts.to_frame())

print("\nEDA mostrado en pantalla.")


Porcentaje de NA por columna:


,pct_na
sector,0.0
monto_financiado,0.0
num_rondas,0.0
tamano_equipo,0.0
exp_fundadores,0.0
presencia_redes,0.0
ubicacion,0.0
inversores_destacados,0.0
descripcion,0.0
tiempo_fundacion,0.0



Conteo de flags de outliers:


,count
is_outlier_monto_financiado,0
is_outlier_tamano_equipo,0
is_outlier_presencia_redes,70



EDA mostrado en pantalla.


# 4. Ingeniería de características
## Objetivo: 
Preparar un dataset modelable con codificación de categóricas, nuevas variables relevantes y escalado opcional, leyendo desde data/processed sin tocar el crudo.

## Componentes: 
Carga del procesado; creación de ratios como ratio_equipo_inversion y exp_media_fundadores; One‑Hot Encoding en sector, ubicación, estado_operativo y antiguedad_bucket; definición de X/y; escalado opcional para modelos sensibles como logística, SVM o KNN.

In [43]:
# 4.1 Carga
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
parquet = PROCESSED_DIR / 'startups_sintetico_1000_processed.parquet'
csv = PROCESSED_DIR / 'startups_sintetico_1000_processed.csv'

dfp = pd.read_parquet(parquet) if parquet.exists() else pd.read_csv(csv)

In [45]:
# 4.2 Variables derivadas adicionales
if set(["tamano_equipo","monto_financiado"]).issubset(dfp.columns):
    ratio = dfp["tamano_equipo"] / dfp["monto_financiado"].where(dfp["monto_financiado"]!=0, np.nan)
    dfp["ratio_equipo_inversion"] = ratio.where(np.isfinite(ratio), 0).fillna(0)

if set(["exp_fundadores","tamano_equipo"]).issubset(dfp.columns):
    exp_med = dfp["exp_fundadores"] / dfp["tamano_equipo"].where(dfp["tamano_equipo"]!=0, np.nan)
    dfp["exp_media_fundadores"] = exp_med.where(np.isfinite(exp_med), 0).fillna(0)

In [46]:
# 4.3 One-Hot Encoding de categóricas clave
cat_cols = [c for c in ["sector","ubicacion","estado_operativo","antiguedad_bucket"] if c in dfp.columns]
dfp_encoded = pd.get_dummies(dfp, columns=cat_cols, drop_first=True, dtype=int)

In [47]:
# 4.4 Definir X e y
if "viabilidad" in dfp_encoded.columns:
    y = dfp_encoded["viabilidad"].astype(int)
else:
    raise ValueError("No se encontró la columna objetivo 'viabilidad'.")

# Variables numéricas base + derivadas
base_num = [c for c in [
    "monto_financiado","num_rondas","tamano_equipo","exp_fundadores",
    "presencia_redes","tiempo_fundacion","log_monto","intensidad_redes",
    "inversores_destacados","antiguedad","ratio_equipo_inversion","exp_media_fundadores"
] if c in dfp_encoded.columns]

# Todas las dummies añadidas
dummy_cols = [c for c in dfp_encoded.columns if any(c.startswith(k+"_") for k in cat_cols)]

feature_cols = base_num + dummy_cols
X = dfp_encoded[feature_cols].copy()

print("Shapes => X:", X.shape, " y:", y.shape)

Shapes => X: (1000, 37)  y: (1000,)


In [ ]:
# 4.5 Escalado
from sklearn.preprocessing import StandardScaler, MinMaxScaler

USE_SCALING = False  # cambiar a True si se usará logística/SVM/KNN
SCALER_KIND = "standard"  # "standard" o "minmax"

if USE_SCALING:
    scaler = StandardScaler() if SCALER_KIND == "standard" else MinMaxScaler()
    X_scaled = scaler.fit_transform(X.values)
    X = pd.DataFrame(X_scaled, columns=feature_cols)
    print("Escalado aplicado:", SCALER_KIND)